In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
import furniture_bench
import gym
import torch

import sys
sys.path.append('/data/pulkitag/data/anthony/repos/research2/furniture-diffusion')
from src.models.vision import get_encoder

In [ ]:
# setup meshcat
from sim_web_visualizer.isaac_visualizer_client import create_isaac_visualizer
mc_vis = create_isaac_visualizer(port=6000, host="localhost", keep_default_viewer=False, max_env=4)

## Test normal env

In [ ]:
env = gym.make(
    "FurnitureSim-v0",
    furniture="one_leg",  # Specifies the type of furniture [lamp | square_table | desk | drawer | cabinet | round_table | stool | chair | one_leg].
    num_envs=16,  # Number of parallel environments.
    resize_img=True,  # If true, images are resized to 224 x 224.
    concat_robot_state=True,  # If true, robot state is concatenated to the observation.
    headless=True,  # If true, simulation runs without GUI.
    compute_device_id=0,
    graphics_device_id=0,
    init_assembled=False,  # If true, the environment is initialized with assembled furniture.
    np_step_out=False,  # If true, env.step() returns Numpy arrays.
    channel_first=False,  # If true, images are returned in channel first format.
    randomness="low",  # Level of randomness in the environment [low | med | high].
    high_random_idx=-1,  # Index of the high randomness level (range: [0-2]). Default -1 will randomly select the index within the range.
    save_camera_input=False,  # If true, the initial camera inputs are saved.
    record=False,  # If true, videos of the wrist and front cameras' RGB inputs are recorded.
    max_env_steps=3000,  # Maximum number of steps per episode.
    act_rot_repr="quat",  # Representation of rotation for action space. Options are 'quat' and 'axis'.
    mc_vis=mc_vis
)

In [ ]:
# Sample action
action = env.action_space.sample()
print("Action shape: ", action.shape)

In [ ]:
env.reset()

# Take a step
obs, reward, done, info = env.step(action)

In [ ]:
obs["color_image2"].device

In [ ]:
obs["robot_state"].shape

In [ ]:
for _ in range(250):
    # Take a step
    action = env.action_space.sample()
    obs, reward, done, info = env.step(action)

In [ ]:
env.reset()

In [ ]:
action = env.action_space.sample()
obs, reward, done, info = env.step(action)

## Test Feature Env

In [ ]:
encoder = get_encoder("dinov2-base")

env = gym.make(
    "FurnitureSimFeature-v0",
    furniture="one_leg",  # Specifies the type of furniture [lamp | square_table | desk | drawer | cabinet | round_table | stool | chair | one_leg].
    num_envs=4,  # Number of parallel environments.
    headless=True,  # If true, simulation runs without GUI.
    encoder=encoder,
    compute_device_id=0,
    graphics_device_id=0,
    init_assembled=False,  # If true, the environment is initialized with assembled furniture.
    randomness="low",  # Level of randomness in the environment [low | med | high].
    high_random_idx=-1,  # Index of the high randomness level (range: [0-2]). Default -1 will randomly select the index within the range.
    save_camera_input=False,  # If true, the initial camera inputs are saved.
    record=False,  # If true, videos of the wrist and front cameras' RGB inputs are recorded.
    max_env_steps=3000,  # Maximum number of steps per episode.
    act_rot_repr="quat",  # Representation of rotation for action space. Options are 'quat' and 'axis'.
)

In [ ]:
obs = env.reset()

In [ ]:
encoder(obs["color_image2"]).shape

In [ ]:
encoder = get_encoder("dinov2-base")

In [ ]:
encoder(torch.zeros(16, 2, 224, 224, 3).reshape(-1, 224, 224, 3)).reshape(
    16, 2, -1
).shape